<h3>모든 파일의 한시간 단위 데이터를 하나의 데이터 프레임으로 합치기</h3>

In [ ]:
#!pip install openpyxl

In [1]:
import openpyxl as op
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os 

In [26]:
src_dir = './2_xlsx_files/'#<----엑셀 파일들이 저장된 폴더명
separate_date = False # 년월일 정보를 분리해서 엑셀에 저장할지 여부
if separate_date:
    output_filename = 'hourly_power(separate_date).xls' # 최종 결과를 저장할 파일명
else:
    output_filename = 'hourly_power(no_separate_date).xls' # 최종 결과를 저장할 파일명

### 사용될 최종 데이터 프레임 초기화

In [27]:
if separate_date:
    final_dataset=pd.DataFrame(columns=['시', '사용량\n(kWh)', 
                                        '최대수요\n(kW)', '무효전력 (kVarh)_지상', 
                                        '무효전력 (kVarh)_진상', 'CO2\n(tCO2)', 
                                        '역률 (%)_지상', '역률 (%)_진상', 
                                        'date', 'year', 'month', 'day'])
else:
    final_dataset=pd.DataFrame(columns=['시', '사용량\n(kWh)', 
                                        '최대수요\n(kW)', '무효전력 (kVarh)_지상', 
                                        '무효전력 (kVarh)_진상', 'CO2\n(tCO2)', 
                                        '역률 (%)_지상', '역률 (%)_진상', 
                                        'date'])

### 모든 파일의 데이터를 최종 데이터 프레임에 추가

In [28]:
hours_per_day = 24
ten_days = 10  # 약, 한달의 1/3
count_valid_days = 0 # 최종 결과에 포함되는 날 수

files = os.listdir(src_dir) # 경로에 있는 모든 파일명 추출

for file_name in files: # 각 파일명에 대해서 ...
    # fname = "경로명 + 파일명"
    fname = os.path.join(src_dir, file_name)
    # 엑셀 파일 읽어서 판다스 데이터프레임에 저장
    df = pd.read_excel(fname, skiprows=[0,1])
    df.columns=['시', '사용량\n(kWh)', '최대수요\n(kW)', 
                '무효전력 (kVarh)_지상', '무효전력 (kVarh)_진상',
                'CO2\n(tCO2)', '역률 (%)_지상', '역률 (%)_진상', 
                '시.1', '사용량\n(kWh).1', '최대수요\n(kW).1', 
                '무효전력 (kVarh)_지상.1', '무효전력 (kVarh)_진상.1', 
                'CO2\n(tCO2).1', '역률 (%)_지상.1', '역률 (%)_진상.1']
    index=[i for i in range(13,63)] # 15분 단위 데이터 인덱스
    df.drop(index, axis=0, inplace=True) # 15분 단위 데이터 삭제
    df.drop([0], axis=0, inplace=True) # NaN이 포함된 칼럼 이름 행 삭제
    dataset_from13=df.iloc[:,[8,9,10,11,12,13,14,15]] # 13시~24시까지 데이터
    dataset_from13.columns=['시', '사용량\n(kWh)', '최대수요\n(kW)', 
                            '무효전력 (kVarh)_지상', '무효전력 (kVarh)_진상',
                            'CO2\n(tCO2)', '역률 (%)_지상', '역률 (%)_진상']
                            # 칼럼 이름을 최종 데이터 프레임의 칼럼 이름과 같게 재설정
    dataset_from0_to12=df.iloc[:,[0,1,2,3,4,5,6,7]] # 1시~12시까지 데이터
    result=pd.concat([dataset_from0_to12, dataset_from13], axis=0) # 하나의 데이터 프레임으로 합체
    
    # 파일명에서 년월일 정보를 추출해서, date 칼럼으로 추가
    curr_date = file_name.split('(')[1].split(')')[0]
    result['date'] = [curr_date]*hours_per_day
    
    if separate_date:
        year = curr_date[0:4]
        month = curr_date[4:6]
        day = curr_date[6:]
        result['year'] = [year]*hours_per_day
        result['month'] = [month]*hours_per_day
        result['day'] = [day]*hours_per_day
    
    # 월 단위로, 전력 총 사용량이 0인 날 수를 계산
    zero_total_power_days = (result['사용량\n(kWh)'] == 0).sum()
    # 월 단위로, 전력 PEAK 수치가 0인 날 수를 계산
    peak_total_power_days = (result['최대수요\n(kW)'] == 0).sum()
    #print(curr_date + " : zeros in total power (%d) and peak power (%d)"%(zero_total_power_days,peak_total_power_days))
    
    # 한달을 기준으로, 전력 사용량 또는 PEAK가 10일 넘게 0을 기록했다면
    # 최종 데이터에 포함하지 않도록...
    if (zero_total_power_days > ten_days) or (peak_total_power_days > ten_days):
        pass
    else:
        final_dataset=pd.concat([final_dataset, result]) #최종 데이터 프레임에 추가
        count_valid_days += 1

# 최종 데이터 프레임의 index 재설정
final_dataset.index=[i for i in range(1, count_valid_days*hours_per_day+1)] 
print('done')

done


In [ ]:
final_dataset # "파일 개수 * 각 파일당 24행" 만큼의 정보가 저장됨

### 결측치 있는지 확인

In [30]:
final_dataset.isnull().sum()#결과가 모두 0 -> 결측치 없음

시                  0
사용량\n(kWh)         0
최대수요\n(kW)         0
무효전력 (kVarh)_지상    0
무효전력 (kVarh)_진상    0
CO2\n(tCO2)        0
역률 (%)_지상          0
역률 (%)_진상          0
date               0
dtype: int64

### 최종 결과를 엑셀 파일로 저장

In [31]:
#!pip install xlwt

In [32]:
final_dataset.to_excel(output_filename, sheet_name='data')

c:\users\daniel\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  """Entry point for launching an IPython kernel.
